In [1]:
import ROOT
import root_numpy as rnpy
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
import pickle
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM

Welcome to ROOTaaS 6.06/08


Using Theano backend.


In [2]:
# build here the keras model
def RNN_classifier():
    model = Sequential()
    
    model.add(LSTM(32, input_shape = (None, 8)))
    
    # make an output layer with just 1 output -> for a binary classification problem: b-jet / not b-jet
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
    return model

In [3]:
def prepare_training_data(jet_list, label):
    # extract the tracks and put them in pt-order, hardest tracks first
    jet_tracks = [cur[-1] for cur in jet_list]
    jet_tracks = [sorted(cur, key = lambda tracks: tracks[0], reverse = True) for cur in jet_tracks]
    
    # zero-pad the track dimension, to make sure all jets fed into the network during training have the same length
    max_tracks = max([len(cur) for cur in jet_tracks])
    padded = [np.vstack([cur, np.full((max_tracks - len(cur), 8), 0, float)]) for cur in jet_tracks]
    
    batch_size = len(padded)
    timestep_size = max_tracks
    jet_dim = 8
    x_train = np.array(padded).reshape(batch_size, timestep_size, jet_dim)
    y_train = np.full((batch_size, 1), label, float) # all are b-jets!
    
    return x_train, y_train, batch_size

In [23]:
batch_size_jets = 120
batch_size_tracks = 520
read_pos_jets = 0
read_pos_tracks = 0
number_chunks = 0
chunks_limit = 20

In [24]:
model = RNN_classifier()

In [25]:
while number_chunks < chunks_limit:
    number_chunks += 1
    
    # read in new chunk of jet and track data
    #d1 = pd.DataFrame(rnpy.root2array("/mnt/t3nfs01/data01/shome/jpata/btv/gc/TagVarExtractor/GCa08e5e237323/TT_TuneCUETP8M1_13TeV-powheg-pythia8/job_0_out.root",
    #                            treename = "tagVars/ttree", start = read_pos_jets, stop = read_pos_jets + batch_size_jets))
    #d2 = pd.DataFrame(rnpy.root2array("/mnt/t3nfs01/data01/shome/jpata/btv/gc/TagVarExtractor/GCa08e5e237323/TT_TuneCUETP8M1_13TeV-powheg-pythia8/job_0_out.root",
    #                            treename = "tagVars/ttree_track", start = read_pos_tracks, stop = read_pos_tracks + batch_size_tracks))
    
    d1 = pd.read_hdf('../0.h5', key = 'jets', start = read_pos_jets, stop = read_pos_jets + batch_size_jets)
    d1 = d1.reset_index(drop=True)
    
    d2 = pd.read_hdf('../0.h5', key = 'tracks', start = read_pos_tracks, stop = read_pos_tracks + batch_size_tracks)
    d2 = d2.reset_index(drop=True)
    
    # break if we reached the end of the file
    if len(d1) < batch_size_jets:
        break;
        
    if len(d2) < batch_size_tracks:
        break;
    
    # figure out where the next chunk should start so that we don't count any jets multiple times
    last_tracks = (int)(d2.tail(1)['Track_jetIndex'].iloc[0]-1)
    last_jet = (int)(d1.tail(1)['Jet_jetIndex'].iloc[0]-1)
    
    # find the latest jet index thatis fully contained in this chunk
    while(len(d2.loc[d2['Track_jetIndex'] == last_tracks]) == 0):
        last_tracks -= 1
        
    if last_tracks > last_jet:
        print("Error: have more tracks than jets! Choose different chunk sizes!")
    
    read_pos_jets += (d1.loc[d1['Jet_jetIndex'] == last_tracks].index[-1] + 1)
    read_pos_tracks += (d2.loc[d2['Track_jetIndex'] == last_tracks].index[-1] + 1)

    # add the track data to the jet list
    d1['track_data'] = pd.np.empty((len(d1.index),0)).tolist()
    
    # iterate over the track list to join jets with the tracks belonging to them
    for irow, row in d2.iterrows():
        # these are the track data of the current track:
        tracks = row[["Track_pt", "Track_eta", "Track_phi", "Track_dxy", "Track_dz", "Track_IP", "Track_IP2D", "Track_length"]].as_matrix()
        jet_index = int(row["Track_jetIndex"])
        if jet_index > last_tracks:
            break
        table_index = d1.loc[d1['Jet_jetIndex'] == jet_index].index[0]

        # append the tracks data to the matching jet in the main table
        d1['track_data'][table_index].append(tracks)
    
    # now divide the jets and put them in separate lists, according to their flavour
    jets_b = []
    jets_l = []
    jets_c = []

    # iterate over the jet list, with already matched tracks
    for irow, row in d1.iterrows():
        jet_index = int(row["Jet_jetIndex"])
        if jet_index > last_tracks:
            break

        flavour = int(row["Jet_flavour"])

        # select the right list this jet belongs to
        if abs(flavour) == 5:
            jets = jets_b
        elif abs(flavour) == 4:
            jets = jets_c
        else:
            jets = jets_l

        # add the new jet to the list, if it contains any resolved tracks
        if len(row["track_data"]) > 0:
            jets += [(row["Jet_pt"], row["Jet_eta"], row["Jet_phi"], row["Jet_mass"], flavour, row["track_data"])]
        
    # now, have sorted jets in three lists, can use them directly for training!
    x_train_b, y_train_b, batch_size_b = prepare_training_data(jets_b, 1)
    x_train_c, y_train_c, batch_size_c = prepare_training_data(jets_c, 0)
    x_train_l, y_train_l, batch_size_l = prepare_training_data(jets_l, 0)
    model.fit(x_train_b, y_train_b, batch_size = batch_size_b, nb_epoch = 5)
    model.fit(x_train_c, y_train_c, batch_size = batch_size_c, nb_epoch = 5)
    model.fit(x_train_l, y_train_l, batch_size = batch_size_l, nb_epoch = 5)

Epoch 1/5
38/38 [==============================] - 0s - loss: 0.9109 - acc: 0.0000e+00
Epoch 2/5
38/38 [==============================] - 0s - loss: 0.8776 - acc: 0.0000e+00
Epoch 3/5
38/38 [==============================] - 0s - loss: 0.8458 - acc: 0.0000e+00
Epoch 4/5
38/38 [==============================] - 0s - loss: 0.8155 - acc: 0.0263
Epoch 5/5
38/38 [==============================] - 0s - loss: 0.7867 - acc: 0.1316
Epoch 1/5
15/15 [==============================] - 0s - loss: 0.4223 - acc: 1.0000
Epoch 2/5
15/15 [==============================] - 0s - loss: 0.4258 - acc: 1.0000
Epoch 3/5
15/15 [==============================] - 0s - loss: 0.4267 - acc: 1.0000
Epoch 4/5
15/15 [==============================] - 0s - loss: 0.4256 - acc: 1.0000
Epoch 5/5
15/15 [==============================] - 0s - loss: 0.4229 - acc: 1.0000
Epoch 1/5
50/50 [==============================] - 0s - loss: 0.6557 - acc: 0.5400
Epoch 2/5
50/50 [==============================] - 0s - loss: 0.6574 - acc:

In [26]:
# save the model after training here
model.save('./RNNtest2.h5')

In [21]:
y_train_c

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]])